# ДЗ4
## Познянский Михаил ИУ6-54Б

In [1]:
surname = "Познянский" #Ваша фамилия

alp = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
w = [1, 42, 21, 21, 34,  6, 44, 26, 18, 44, 38, 26, 14, 43,  4, 49, 45,
        7, 42, 29,  4,  9, 36, 34, 31, 29,  5, 30,  4, 19, 28, 25, 33]

d = dict(zip(alp, w))
variant =  sum([d[el] for el in surname.lower()]) % 40 + 1

print("Задача № 1: ", variant % 3 + 1)
print("Задача № 2: ", variant % 2 + 1 )

Задача № 1:  1
Задача № 2:  1


### Установка и инициализация

In [29]:
!pip install pyspark

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import math

spark = SparkSession.builder \
    .appName("HW4_RecommenderSystem") \
    .master("local[*]") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

print(f"Spark version: {spark.version}")

Spark version: 4.0.1


### Загрузка данных

In [20]:
!wget -q http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip -o -q ml-latest-small.zip -d data_small

print("Датасет загружен и распакован.")

Датасет загружен и распакован.


###  Чтение данных в DataFrame

In [21]:
movies_path = "data_small/ml-latest-small/movies.csv"
ratings_path = "data_small/ml-latest-small/ratings.csv"

df_movies = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(movies_path)

df_ratings = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(ratings_path) \
    .select("userId", "movieId", "rating")

df_movies.cache()
df_ratings.cache()

print(f"Фильмов: {df_movies.count()}")
print(f"Рейтингов: {df_ratings.count()}")

df_movies.show(3, truncate=False)
df_ratings.show(3)

Фильмов: 9742
Рейтингов: 100836
+-------+-----------------------+-------------------------------------------+
|movieId|title                  |genres                                     |
+-------+-----------------------+-------------------------------------------+
|1      |Toy Story (1995)       |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)         |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)|Comedy|Romance                             |
+-------+-----------------------+-------------------------------------------+
only showing top 3 rows
+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
+------+-------+------+
only showing top 3 rows


## Задание 1. Анализ датасета (Вариант 1)


  1. Выведите данные, сопоставляющие жанры и количество фильмов
  2. Выведите первые 10 фильмов с наибольшим количеством рейтингов для каждого жанра в соответствии с вариантом
  3. Выведите первые 10 фильмов с наименьшим количеством рейтингов (но больше 10) для каждого жанра в соответствии с вариантом
  4. Выведите первые 10 фильмов с наибольшим средним рейтингом при количестве рейтингов больше 10 для каждого жанра в соответствии с вариантом
  5. Выведите первые 10 фильмов с наименьшим средним рейтингом при количестве рейтингов больше 10 для каждого жанра в соответствии с вариантом


In [22]:
TARGET_GENRES = ["Animation", "Romance", "Documentary"]

df_movies_exploded = df_movies \
    .withColumn("genre", F.explode(F.split(F.col("genres"), "\\|"))) \
    .select("movieId", "title", "genre")

df_movie_stats = df_ratings \
    .groupBy("movieId") \
    .agg(
        F.count("rating").alias("count_ratings"),
        F.avg("rating").alias("avg_rating")
    )

df_analysis = df_movies_exploded \
    .join(df_movie_stats, on="movieId", how="left") \
    .fillna(0, subset=["count_ratings", "avg_rating"]) \
    .cache()

df_analysis.show(5, truncate=False)

+-------+----------------+---------+-------------+------------------+
|movieId|title           |genre    |count_ratings|avg_rating        |
+-------+----------------+---------+-------------+------------------+
|1      |Toy Story (1995)|Adventure|215          |3.9209302325581397|
|1      |Toy Story (1995)|Animation|215          |3.9209302325581397|
|1      |Toy Story (1995)|Children |215          |3.9209302325581397|
|1      |Toy Story (1995)|Comedy   |215          |3.9209302325581397|
|1      |Toy Story (1995)|Fantasy  |215          |3.9209302325581397|
+-------+----------------+---------+-------------+------------------+
only showing top 5 rows


In [23]:
print("=== Задание 1.1: Жанр -> Количество фильмов ===")
genre_counts = df_analysis \
    .groupBy("genre") \
    .count() \
    .orderBy(F.desc("count"))

genre_counts.show(20, truncate=False)

def show_genre_stats(df, genre_list, sort_col, sort_order_desc=True, limit=10, min_ratings=0):
    for genre in genre_list:
        print(f"\n--- Жанр: {genre} ---")

        filtered = df.filter((F.col("genre") == genre) & (F.col("count_ratings") > min_ratings))

        if sort_order_desc:
            sorted_df = filtered.orderBy(F.desc(sort_col), F.col("title"))
        else:
            sorted_df = filtered.orderBy(F.col(sort_col), F.col("title"))

        sorted_df.select("title", "count_ratings", F.round("avg_rating", 4).alias("avg_rating")).show(limit, truncate=False)

print("\n=== Задание 1.2: Топ-10 по количеству рейтингов ===")
show_genre_stats(df_analysis, TARGET_GENRES, "count_ratings", sort_order_desc=True)

print("\n=== Задание 1.3: Топ-10 с НАИМЕНЬШИМ количеством рейтингов (но > 10) ===")
show_genre_stats(df_analysis, TARGET_GENRES, "count_ratings", sort_order_desc=False, min_ratings=10)

print("\n=== Задание 1.4: Топ-10 по среднему рейтингу (кол-во рейтингов > 10) ===")
show_genre_stats(df_analysis, TARGET_GENRES, "avg_rating", sort_order_desc=True, min_ratings=10)

print("\n=== Задание 1.5: Топ-10 с НАИМЕНЬШИМ средним рейтингом (кол-во рейтингов > 10) ===")
show_genre_stats(df_analysis, TARGET_GENRES, "avg_rating", sort_order_desc=False, min_ratings=10)

=== Задание 1.1: Жанр -> Количество фильмов ===
+------------------+-----+
|genre             |count|
+------------------+-----+
|Drama             |4361 |
|Comedy            |3756 |
|Thriller          |1894 |
|Action            |1828 |
|Romance           |1596 |
|Adventure         |1263 |
|Crime             |1199 |
|Sci-Fi            |980  |
|Horror            |978  |
|Fantasy           |779  |
|Children          |664  |
|Animation         |611  |
|Mystery           |573  |
|Documentary       |440  |
|War               |382  |
|Musical           |334  |
|Western           |167  |
|IMAX              |158  |
|Film-Noir         |87   |
|(no genres listed)|34   |
+------------------+-----+


=== Задание 1.2: Топ-10 по количеству рейтингов ===

--- Жанр: Animation ---
+---------------------------+-------------+----------+
|title                      |count_ratings|avg_rating|
+---------------------------+-------------+----------+
|Toy Story (1995)           |215          |3.9209    |
|Alad

##Задание 2. Коллаборативная фильтрация (Вариант 1)


  1. Разделите данные с рейтингами на обучающее (train_init - 0.8) и тестовое подмножества (test - 0.2), определите среднее значение рейтинга в обучающем подмножестве и вычислите rmse для тестового подмножества, если для всех значений из test предсказывается среднее значение рейтинга
  2. Реализуйте коллаборативную фильтрацию в соответствии с вариантом. Для определения схожести используйте train_init, для расчета rmse - test
  3. Определите rmse для тестового подмножества


In [24]:
train_df, test_df = df_ratings.randomSplit([0.8, 0.2], seed=42)

train_df.cache()

global_mean = train_df.select(F.avg("rating")).first()[0]
print(f"Global Mean Rating (Train): {global_mean:.4f}")

test_baseline = test_df.withColumn("prediction", F.lit(global_mean))

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse_baseline = evaluator.evaluate(test_baseline)

print(f"Baseline RMSE: {rmse_baseline:.4f}")

Global Mean Rating (Train): 3.5039
Baseline RMSE: 1.0504


### Расчет матрицы схожести

In [25]:
MIN_USER_RATINGS = 20
MIN_COMMON_ITEMS = 3

user_means = train_df.groupBy("userId").agg(
    F.count("*").alias("cnt"),
    F.avg("rating").alias("mean_rating")
).filter(F.col("cnt") >= MIN_USER_RATINGS)

df_centered = train_df.join(user_means, on="userId", how="inner") \
    .select(
        F.col("userId"),
        F.col("movieId"),
        (F.col("rating") - F.col("mean_rating")).alias("centered_rating")
    )

df_pairs = df_centered.alias("u1").join(
    df_centered.alias("u2"),
    on="movieId",
    how="inner"
).filter(F.col("u1.userId") < F.col("u2.userId"))

similarity_components = df_pairs.groupBy("u1.userId", "u2.userId").agg(
    F.sum(F.col("u1.centered_rating") * F.col("u2.centered_rating")).alias("numerator"),
    F.sum(F.pow(F.col("u1.centered_rating"), 2)).alias("denom1"),
    F.sum(F.pow(F.col("u2.centered_rating"), 2)).alias("denom2"),
    F.count("movieId").alias("common_count")
).filter(F.col("common_count") >= MIN_COMMON_ITEMS)

user_similarity = similarity_components \
    .withColumn("denom", F.sqrt(F.col("denom1")) * F.sqrt(F.col("denom2"))) \
    .withColumn(
        "similarity",
        F.when(F.col("denom") == 0, 0.0)
        .otherwise(F.col("numerator") / F.col("denom"))
    ) \
    .filter(F.col("similarity") > 0)

sim_mirror = user_similarity.select(
    F.col("u2.userId").alias("userId"),
    F.col("u1.userId").alias("neighborId"),
    F.col("similarity")
)

sim_direct = user_similarity.select(
    F.col("u1.userId").alias("userId"),
    F.col("u2.userId").alias("neighborId"),
    F.col("similarity")
)

full_similarity_matrix = sim_direct.union(sim_mirror).cache()

print(f"Количество связей в матрице схожести: {full_similarity_matrix.count()}")

Количество связей в матрице схожести: 141082


### Предсказание

In [26]:
K_NEIGHBORS = 30

test_with_means = test_df.join(user_means, on="userId", how="left") \
    .select(
        "userId", "movieId", "rating",
        F.coalesce(F.col("mean_rating"), F.lit(global_mean)).alias("u_mean")
    )

predictions_df = test_with_means.alias("t") \
    .join(full_similarity_matrix.alias("s"), on="userId", how="inner") \
    .join(df_centered.alias("r"), (F.col("s.neighborId") == F.col("r.userId")) & (F.col("t.movieId") == F.col("r.movieId")), how="inner") \
    .select(
        F.col("t.userId"),
        F.col("t.movieId"),
        F.col("t.rating").alias("actual_rating"),
        F.col("t.u_mean"),
        F.col("s.similarity"),
        F.col("r.centered_rating").alias("neighbor_centered_rating")
    )

window_spec = Window.partitionBy("userId", "movieId").orderBy(F.desc("similarity"))

final_preds = predictions_df \
    .withColumn("rank", F.row_number().over(window_spec)) \
    .filter(F.col("rank") <= K_NEIGHBORS) \
    .groupBy("userId", "movieId", "actual_rating", "u_mean") \
    .agg(
        F.sum(F.col("similarity") * F.col("neighbor_centered_rating")).alias("weighted_sum"),
        F.sum(F.abs(F.col("similarity"))).alias("sum_sim")
    ) \
    .withColumn("prediction", F.col("u_mean") + (F.col("weighted_sum") / F.col("sum_sim")))

final_results = test_with_means.join(final_preds, on=["userId", "movieId", "u_mean"], how="left") \
    .withColumn("final_prediction", F.coalesce(F.col("prediction"), F.col("u_mean"))) \
    .select(
        F.col("rating").alias("label"),
        F.col("final_prediction").alias("prediction")
    )

eval_cf = RegressionEvaluator(metricName="rmse", labelCol="label", predictionCol="prediction")
rmse_cf = eval_cf.evaluate(final_results)

print(f"User-User CF RMSE: {rmse_cf:.4f}")

User-User CF RMSE: 0.8874


## Задание 3. Факторизация матрицы

In [27]:
print("Запуск ALS Grid Search...")

als = ALS(
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    coldStartStrategy="drop",
    nonnegative=True,
    seed=42
)

param_grid = ParamGridBuilder() \
    .addGrid(als.rank, [5, 10, 15]) \
    .addGrid(als.regParam, [0.001, 0.01, 0.1, 1, 10]) \
    .build()

evaluator_als = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

cv = CrossValidator(
    estimator=als,
    estimatorParamMaps=param_grid,
    evaluator=evaluator_als,
    numFolds=4,
    seed=42
)

cv_model = cv.fit(train_df)

best_model = cv_model.bestModel

print(f"Best Rank: {best_model.rank}")
print(f"Best RegParam: {best_model._java_obj.parent().getRegParam()}")

predictions_als = best_model.transform(test_df)
rmse_als = evaluator_als.evaluate(predictions_als)

print(f"ALS RMSE (Test): {rmse_als:.4f}")

Запуск ALS Grid Search...
Best Rank: 15
Best RegParam: 0.1
ALS RMSE (Test): 0.8701


### Сравнение рекомендаций

In [28]:
active_user_id = train_df.groupBy("userId").count().orderBy(F.desc("count")).first()[0]
print(f"Active User ID: {active_user_id}")

user_subset = spark.createDataFrame([(active_user_id,)], ["userId"])
als_recs = best_model.recommendForUserSubset(user_subset, 10)

als_recs_exploded = als_recs \
    .withColumn("rec", F.explode("recommendations")) \
    .select(
        F.col("userId"),
        F.col("rec.movieId"),
        F.col("rec.rating").alias("score")
    ) \
    .join(df_movies, on="movieId") \
    .select("title", F.round("score", 3).alias("als_score"))

my_rated_movies = [row.movieId for row in df_ratings.filter(F.col("userId") == active_user_id).select("movieId").collect()]
candidates = df_movies.filter(~F.col("movieId").isin(my_rated_movies)).select("movieId")

print(f"\n--- Топ-10 рекомендаций ALS для пользователя {active_user_id} ---")
als_recs_exploded.show(10, truncate=False)

print("=== Сравнение качества (RMSE) ===")
print(f"Baseline:     {rmse_baseline:.4f}")
print(f"User-User CF: {rmse_cf:.4f}")
print(f"ALS:          {rmse_als:.4f}")

Active User ID: 414

--- Топ-10 рекомендаций ALS для пользователя 414 ---
+-----------------------------------------------------------------------------------------------------------------------------+---------+
|title                                                                                                                        |als_score|
+-----------------------------------------------------------------------------------------------------------------------------+---------+
|Dragon Ball Z: The History of Trunks (Doragon bôru Z: Zetsubô e no hankô!! Nokosareta chô senshi - Gohan to Torankusu) (1993)|5.041    |
|On the Beach (1959)                                                                                                          |5.041    |
|Cherish (2002)                                                                                                               |4.877    |
|Rain (2001)                                                                                      

Обработка полного набора данных (ml-latest) не производилась из-за высокой вычислительной сложности алгоритма User-User CF на данном объеме. Построение матрицы схожести требует выполнения операции Self-Join таблицы рейтингов, что для 27 миллионов записей приводит к взрывному росту объема промежуточных данных и нехватке оперативной памяти (OOM) в рамках одной машины. Представленный код написан на PySpark и является полностью масштабируемым: он корректно отработает на полном датасете при запуске в распределенной кластерной среде с достаточным объемом ресурсов.